## we will be looking at databases with SQLite

https://juliadatabases.org/SQLite.jl/stable/

In [3]:
import Pkg
#Pkg.add("SQLite")
#Pkg.add("DataFrames")
Pkg.add("Tables")
#Pkg.add("CSV")

Pkg.add("HTTP")
Pkg.add("JSON")
Pkg.add("Gumbo")
Pkg.add("Cascadia")

Pkg.status("SQLite")

   Resolving package versions...
    Updating `~/.julia/environments/v1.11/Project.toml`
  [bd369af6] + Tables v1.12.0
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
    Updating `~/.julia/environments/v1.11/Project.toml`
  [cd3eb016] + HTTP v1.10.15
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
    Updating `~/.julia/environments/v1.11/Project.toml`
  [682c06a0] + JSON v0.21.4
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
   Installed Gumbo_jll ─ v0.10.2+0
   Installed Gumbo ───── v0.8.3
    Updating `~/.julia/environments/v1.11/Project.toml`
  [708ec375] + Gumbo v0.8.3
    Updating `~/.julia/environments/v1.11/Manifest.toml`
  [1520ce14] + AbstractTrees v0.4.5
  [708ec375] + Gumbo v0.8.3
  [528830af] + Gumbo_jll v0.10.2+0
Precompiling project...
   1199.1 ms  ✓ Gumbo_jll
    824.6 ms  ✓ Gumbo
  2 dependencies successfully precompiled in 3 seconds

Status 

    899.7 ms  ✓ Cascadia
  1 dependency successfully precompiled in 2 seconds. 483 already precompiled.


`~/.julia/environments/v1.11/Project.toml`
  [0aa819cd] SQLite v1.6.1


In [4]:
using SQLite
using DataFrames
using Tables
using CSV

using HTTP
using JSON

using Gumbo
using Cascadia

using Statistics

In [5]:
# list files in the current directory as this notebook
run(`ls`)

01SQLite.ipynb
02SQLIte.ipynb
Chinook.ipynb
customer_data.sqlite
newCustomers.csv


Process(`ls`, ProcessExited(0))

In [7]:
#create a new database file
db = SQLite.DB("my_database.db")

SQLite.DB("my_database.db")

In [8]:
run(`ls`)

01SQLite.ipynb
02SQLIte.ipynb
Chinook.ipynb
customer_data.sqlite
my_database.db
newCustomers.csv


Process(`ls`, ProcessExited(0))

In [10]:
#create a simple table
SQLite.execute(db, """
CREATE TABLE IF NOT EXISTS people (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    age INTEGER
)
""")

101

In [11]:
# 'Query' the table
for row in DBInterface.execute(db, "SELECT * FROM people")
    println(row)
end

In [12]:
#insert data into the table, notice that the id key is not explicitly inserted
SQLite.execute(db, "INSERT INTO people (name, age) VALUES (?, ?)", ("Alan", 30))
SQLite.execute(db, "INSERT INTO people (name, age) VALUES (?, ?)", ("Bob", 25))

101

In [13]:
DBInterface.execute(db, "INSERT INTO people (name, age) VALUES (?, ?)", ("Cat", 50))


SQLite.Query{false}(SQLite.Stmt(SQLite.DB("my_database.db"), Base.RefValue{Ptr{SQLite.C.sqlite3_stmt}}(Ptr{SQLite.C.sqlite3_stmt} @0x000000002f39bf28), Dict{Int64, Any}(2 => 50, 1 => "Cat")), Base.RefValue{Int32}(101), Symbol[], Type[], Dict{Symbol, Int64}(), Base.RefValue{Int64}(0))

In [14]:
for row in DBInterface.execute(db, "SELECT * FROM people")
    println(row)
end

SQLite.Row{false}:
 :id     1
 :name    "Alan"
 :age   30
SQLite.Row{false}:
 :id     2
 :name    "Bob"
 :age   25
SQLite.Row{false}:
 :id     3
 :name    "Cat"
 :age   50


In [15]:
#ypdate data in the table
SQLite.execute(db, "UPDATE people SET age = ? WHERE name = ?", (35, "Alan"))

101

In [16]:
for row in DBInterface.execute(db, "SELECT * FROM people")
    println(row)
end

SQLite.Row{false}:
 :id     1
 :name    "Alan"
 :age   35
SQLite.Row{false}:
 :id     2
 :name    "Bob"
 :age   25
SQLite.Row{false}:
 :id     3
 :name    "Cat"
 :age   50


In [17]:
#delete a record from the table
SQLite.execute(db, "DELETE FROM people WHERE name = ?", ("Bob",))

101

In [18]:
for row in DBInterface.execute(db, "SELECT * FROM people")
    println(row)
end

SQLite.Row{false}:
 :id     1
 :name    "Alan"
 :age   35
SQLite.Row{false}:
 :id     3
 :name    "Cat"
 :age   50


In [19]:
# Insert multiple rows in a single statement
SQLite.execute(db, """
INSERT INTO people (name, age) VALUES 
    ('Charlie', 40),
    ('David', 28),
    ('Eve', 35)
""")

101

In [21]:
result_df = DataFrame( DBInterface.execute(db, "SELECT * FROM people") )
result_df

Row,id,name,age
,Int64,String,Int64
1,1,Alan,35
2,3,Cat,50
3,4,Charlie,40
4,5,David,28
5,6,Eve,35


In [22]:
#adding a new column 'state' with a default value
SQLite.execute(db, """
ALTER TABLE people
ADD COLUMN state TEXT DEFAULT 'Unknown'
""")

101

In [23]:
result_df = DBInterface.execute(db, "SELECT * FROM people") |> DataFrame
result_df

Row,id,name,age,state
,Int64,String,Int64,String
1,1,Alan,35,Unknown
2,3,Cat,50,Unknown
3,4,Charlie,40,Unknown
4,5,David,28,Unknown
5,6,Eve,35,Unknown


In [24]:
#update all those over 30 to be state FL
SQLite.execute(db, "UPDATE people SET state = 'FL' WHERE age > 30")

101

In [26]:
[1,2,3] |> sum |> x-> x^2

36

In [25]:
result_df = DBInterface.execute(db, "SELECT * FROM people") |> DataFrame
result_df

Row,id,name,age,state
,Int64,String,Int64,String
1,1,Alan,35,FL
2,3,Cat,50,FL
3,4,Charlie,40,FL
4,5,David,28,Unknown
5,6,Eve,35,FL


In [27]:
#number of people in FL
people_in_fl = DBInterface.execute(db, "SELECT COUNT(*) FROM people WHERE state = 'FL'") |> DataFrame

Row,COUNT(*)
,Int64
1,4


In [28]:
average_age_by_state = DBInterface.execute(db, "SELECT state, AVG(age) as average_age FROM people GROUP BY state") |> DataFrame

Row,state,average_age
,String,Float64
1,FL,40.0
2,Unknown,28.0


In [29]:
#Find this using DataFrames directly
result_df = DBInterface.execute(db, "SELECT * FROM people") |> DataFrame

Row,id,name,age,state
,Int64,String,Int64,String
1,1,Alan,35,FL
2,3,Cat,50,FL
3,4,Charlie,40,FL
4,5,David,28,Unknown
5,6,Eve,35,FL


In [30]:
fl_count = sum(result_df.state .== "FL") #number of FL people

4

In [31]:
average_age_by_state_df = combine(groupby(result_df, :state), :age => mean => :average_age)

Row,state,average_age
,String,Float64
1,FL,40.0
2,Unknown,28.0


In [32]:
groupby(result_df, :state)

Row,id,name,age,state
,Int64,String,Int64,String
1,1,Alan,35,FL
2,3,Cat,50,FL
3,4,Charlie,40,FL
4,6,Eve,35,FL
Row,id,name,age,state
,Int64,String,Int64,String
1,5,David,28,Unknown


In [34]:
run(`ls`)

01SQLite.ipynb
02SQLIte.ipynb
Chinook.ipynb
customer_data.sqlite
my_database.db
newCustomers.csv


Process(`ls`, ProcessExited(0))

# online store scenario

In [35]:
#create the 'customers' table
SQLite.execute(db, """
CREATE TABLE IF NOT EXISTS customers (
    customer_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    state TEXT NOT NULL
)
""")

101

In [37]:
#create the 'purchases' table
SQLite.execute(db, """
CREATE TABLE IF NOT EXISTS purchases (
    purchase_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    amount REAL,
    FOREIGN KEY (customer_id) REFERENCES customers (customer_id)
)
""")

101

In [38]:
customers_data = [
    (1, "Alice", "FL"),
    (2, "Bob", "NY"),
    (3, "Charlie", "FL"),
    (4, "David", "CA"),
    (5, "Eve", "TX")
]

5-element Vector{Tuple{Int64, String, String}}:
 (1, "Alice", "FL")
 (2, "Bob", "NY")
 (3, "Charlie", "FL")
 (4, "David", "CA")
 (5, "Eve", "TX")

In [39]:
for customer in customers_data
    println(customer)
    SQLite.execute(db, "INSERT INTO customers (customer_id, name, state) VALUES (?, ?, ?)", customer)
end

(1, "Alice", "FL")
(2, "Bob", "NY")
(3, "Charlie", "FL")
(4, "David", "CA")
(5, "Eve", "TX")


In [40]:
DBInterface.execute(db, "SELECT * FROM customers") |> DataFrame

Row,customer_id,name,state
,Int64,String,String
1,1,Alice,FL
2,2,Bob,NY
3,3,Charlie,FL
4,4,David,CA
5,5,Eve,TX


In [41]:
# Insert data into 'purchases'
purchases_data = [
    (1, 2, 150.00),  # Bob has one purchase
    (2, 3, 200.00),  # Charlie has two purchases
    (3, 3, 300.00),
    (4, 4, 50.00),   # David has two purchases
    (5, 4, 75.00),
    (6, 5, 100.00),  # Eve has two purchases
    (7, 5, 80.00)
]

7-element Vector{Tuple{Int64, Int64, Float64}}:
 (1, 2, 150.0)
 (2, 3, 200.0)
 (3, 3, 300.0)
 (4, 4, 50.0)
 (5, 4, 75.0)
 (6, 5, 100.0)
 (7, 5, 80.0)

In [42]:
for purchase in purchases_data
    SQLite.execute(db, "INSERT INTO purchases (purchase_id, customer_id, amount) VALUES (?, ?, ?)", purchase)
end

In [43]:
DBInterface.execute(db, "SELECT * FROM purchases") |> DataFrame

Row,purchase_id,customer_id,amount
,Int64,Int64,Float64
1,1,2,150.0
2,2,3,200.0
3,3,3,300.0
4,4,4,50.0
5,5,4,75.0
6,6,5,100.0
7,7,5,80.0


- Alice has no purchases.
- Bob has a single purchase.
- Charlie, David, and Eve have a couple of purchases each

do LEFT JOIN

In [44]:
joined_data = DBInterface.execute(db, """
SELECT customers.name, customers.state, purchases.amount
FROM customers
LEFT JOIN purchases ON customers.customer_id = purchases.customer_id
""") |> DataFrame

Row,name,state,amount
,String,String,Float64?
1,Alice,FL,missing
2,Bob,NY,150.0
3,Charlie,FL,200.0
4,Charlie,FL,300.0
5,David,CA,50.0
6,David,CA,75.0
7,Eve,TX,80.0
8,Eve,TX,100.0


do an INNER JOIN

In [45]:
joined_data = DBInterface.execute(db, """
SELECT customers.name, customers.state, purchases.amount
FROM customers
JOIN purchases ON customers.customer_id = purchases.customer_id
""") |> DataFrame

Row,name,state,amount
,String,String,Float64
1,Bob,NY,150.0
2,Charlie,FL,200.0
3,Charlie,FL,300.0
4,David,CA,50.0
5,David,CA,75.0
6,Eve,TX,100.0
7,Eve,TX,80.0


In [47]:
# list all the tables
SQLite.tables(db)

3-element Vector{SQLite.DBTable}:
 SQLite.DBTable("people", Tables.Schema:
 :id     Union{Missing, Int64}
 :name   Union{Missing, String}
 :age    Union{Missing, Int64}
 :state  Union{Missing, String})
 SQLite.DBTable("customers", Tables.Schema:
 :customer_id  Union{Missing, Int64}
 :name         Union{Missing, String}
 :state        Union{Missing, String})
 SQLite.DBTable("purchases", Tables.Schema:
 :purchase_id  Union{Missing, Int64}
 :customer_id  Union{Missing, Int64}
 :amount       Union{Missing, Float64})

In [48]:
customers_df = DataFrame(
    customer_id = 41:44,
    name = ["Todd", "Simon", "Robert", "Tom"],
    state = ["FL", "NY", "FL", "CA"]
)

println(customers_df)

4×3 DataFrame
 Row │ customer_id  name    state  
     │ Int64        String  String 
─────┼─────────────────────────────
   1 │          41  Todd    FL
   2 │          42  Simon   NY
   3 │          43  Robert  FL
   4 │          44  Tom     CA


In [50]:
#save the DataFrame to a CSV file
CSV.write("newCustomers.csv", customers_df)

"newCustomers.csv"

In [54]:
#read the CSV file as if it was new
new_customers_df = CSV.read("./newCustomers.csv", DataFrame)

Row,customer_id,name,state
,Int64,String7,String3
1,41,Todd,FL
2,42,Simon,NY
3,43,Robert,FL
4,44,Tom,CA


In [56]:
# Insert the data from the DataFrame into the 'customers' table
for row in eachrow(new_customers_df)
    SQLite.execute(db, "INSERT INTO customers (customer_id, name, state) VALUES (?, ?, ?)", 
        (row.customer_id, row.name, row.state))
end

In [57]:
DBInterface.execute(db, "SELECT * FROM customers") |> DataFrame

Row,customer_id,name,state
,Int64,String,String
1,1,Alice,FL
2,2,Bob,NY
3,3,Charlie,FL
4,4,David,CA
5,5,Eve,TX
6,41,Todd,FL
7,42,Simon,NY
8,43,Robert,FL
9,44,Tom,CA


In [58]:
DBInterface.execute(db, "DELETE FROM customers WHERE name='Robert'")

DBInterface.execute(db, "SELECT * FROM customers") |> DataFrame

Row,customer_id,name,state
,Int64,String,String
1,1,Alice,FL
2,2,Bob,NY
3,3,Charlie,FL
4,4,David,CA
5,5,Eve,TX
6,41,Todd,FL
7,42,Simon,NY
8,44,Tom,CA


In [59]:
DBInterface.execute(db, "SELECT * FROM purchases") |> DataFrame

Row,purchase_id,customer_id,amount
,Int64,Int64,Float64
1,1,2,150.0
2,2,3,200.0
3,3,3,300.0
4,4,4,50.0
5,5,4,75.0
6,6,5,100.0
7,7,5,80.0


In [60]:
DBInterface.execute(db, "DELETE FROM purchases WHERE amount < 100") |> DataFrame

DBInterface.execute(db, "SELECT * FROM purchases") |> DataFrame

Row,purchase_id,customer_id,amount
,Int64,Int64,Float64
1,1,2,150.0
2,2,3,200.0
3,3,3,300.0
4,6,5,100.0


In [61]:
DBInterface.execute(db, "INSERT INTO purchases (purchase_id,customer_id,amount) VALUES (?, ?, ?)", (10,1,444))

DBInterface.execute(db, "SELECT * FROM purchases") |> DataFrame

Row,purchase_id,customer_id,amount
,Int64,Int64,Float64
1,1,2,150.0
2,2,3,200.0
3,3,3,300.0
4,6,5,100.0
5,10,1,444.0


In [67]:
DBInterface.execute(db, "SELECT * FROM purchases") |> DataFrame

Row,purchase_id,customer_id,amount
,Int64,Int64,Float64
1,1,2,150.0
2,2,3,200.0
3,3,3,300.0
4,6,5,100.0
5,10,1,444.0
6,11,1,555.0


In [68]:
purchases = [(112,1,555), (212,1,666), (213,1,777)]

[ DBInterface.execute(db, "INSERT INTO purchases (purchase_id,customer_id,amount) VALUES (?, ?, ?)", pp) for pp in purchases ]


3-element Vector{SQLite.Query{false}}:
 SQLite.Query{false}(SQLite.Stmt(SQLite.DB("my_database.db"), Base.RefValue{Ptr{SQLite.C.sqlite3_stmt}}(Ptr{SQLite.C.sqlite3_stmt} @0x000000002f398238), Dict{Int64, Any}(2 => 1, 3 => 555, 1 => 112)), Base.RefValue{Int32}(101), Symbol[], Type[], Dict{Symbol, Int64}(), Base.RefValue{Int64}(0))
 SQLite.Query{false}(SQLite.Stmt(SQLite.DB("my_database.db"), Base.RefValue{Ptr{SQLite.C.sqlite3_stmt}}(Ptr{SQLite.C.sqlite3_stmt} @0x000000002f3978d8), Dict{Int64, Any}(2 => 1, 3 => 666, 1 => 212)), Base.RefValue{Int32}(101), Symbol[], Type[], Dict{Symbol, Int64}(), Base.RefValue{Int64}(0))
 SQLite.Query{false}(SQLite.Stmt(SQLite.DB("my_database.db"), Base.RefValue{Ptr{SQLite.C.sqlite3_stmt}}(Ptr{SQLite.C.sqlite3_stmt} @0x000000002f399048), Dict{Int64, Any}(2 => 1, 3 => 777, 1 => 213)), Base.RefValue{Int32}(101), Symbol[], Type[], Dict{Symbol, Int64}(), Base.RefValue{Int64}(0))

In [69]:
SQLite.tables(db)

3-element Vector{SQLite.DBTable}:
 SQLite.DBTable("people", Tables.Schema:
 :id     Union{Missing, Int64}
 :name   Union{Missing, String}
 :age    Union{Missing, Int64}
 :state  Union{Missing, String})
 SQLite.DBTable("customers", Tables.Schema:
 :customer_id  Union{Missing, Int64}
 :name         Union{Missing, String}
 :state        Union{Missing, String})
 SQLite.DBTable("purchases", Tables.Schema:
 :purchase_id  Union{Missing, Int64}
 :customer_id  Union{Missing, Int64}
 :amount       Union{Missing, Float64})

In [70]:
SQLite.drop!(db, "people")

101

In [71]:
SQLite.tables(db)

2-element Vector{SQLite.DBTable}:
 SQLite.DBTable("customers", Tables.Schema:
 :customer_id  Union{Missing, Int64}
 :name         Union{Missing, String}
 :state        Union{Missing, String})
 SQLite.DBTable("purchases", Tables.Schema:
 :purchase_id  Union{Missing, Int64}
 :customer_id  Union{Missing, Int64}
 :amount       Union{Missing, Float64})

In [77]:
for table in SQLite.tables(db)
    println(table) 

    println( typeof(table))
end

SQLite.DBTable("customers", Tables.Schema:
 :customer_id  Union{Missing, Int64}
 :name         Union{Missing, String}
 :state        Union{Missing, String})
SQLite.DBTable
SQLite.DBTable("purchases", Tables.Schema:
 :purchase_id  Union{Missing, Int64}
 :customer_id  Union{Missing, Int64}
 :amount       Union{Missing, Float64})
SQLite.DBTable


In [78]:
SQLite.tables(db)[1]

SQLite.DBTable("customers", Tables.Schema:
 :customer_id  Union{Missing, Int64}
 :name         Union{Missing, String}
 :state        Union{Missing, String})

In [80]:
fieldnames( typeof( SQLite.tables(db)[1] ) )

(:name, :schema)

In [81]:
SQLite.tables(db)[1].name

"customers"

In [54]:
urlstr = "https://forecast.weather.gov/MapClick.php?CityName=Orlando&state=FL&site=MLB&textField1=28.5047&textField2=-81.3742&e=0"

"https://forecast.weather.gov/MapClick.php?CityName=Orlando&state=FL&site=MLB&textField1=28.5047&textField2=-81.3742&e=0"

In [55]:
response = HTTP.get(urlstr)

HTTP.Messages.Response:
"""
HTTP/1.1 200 OK
Server: Apache
X-NIDS-ServerID: www9.md
X-UA-Compatible: IE=Edge
Access-Control-Allow-Origin: *
Content-Type: text/html; charset=UTF-8
Vary: Accept-Encoding
Content-Encoding: gzip
Cache-Control: max-age=669
Expires: Mon, 13 Nov 2023 22:02:56 GMT
Date: Mon, 13 Nov 2023 21:51:47 GMT
Content-Length: 11828
Connection: keep-alive
Strict-Transport-Security: max-age=31536000 ; includeSubDomains ; preload

<!DOCTYPE html>
<html class="no-js">
    <head>
        <!-- Meta -->
        <meta name="viewport" content="width=device-width">
        <link rel="schema.DC" href="http://purl.org/dc/elements/1.1/" /><title>National Weather Service</title><meta name="DC.title" content="National Weather Service" /><meta name="DC.description" content="NOAA National Weather Service National Weather Service" /><meta name="DC.creator" content="US Department of Commerce, NOAA, National Weather Service" /><meta name="DC.date.created" scheme="ISO8601" content="" /><meta 

In [56]:
response = String( response )

"HTTP/1.1 200 OK\r\nServer: Apache\r\nX-NIDS-ServerID: www9.md\r\nX-UA-Compatible: IE=Edge\r\nAccess-Control-Allow-Origin: *\r\nContent-Type: text/html; charset=UTF-8\r\nVary: Accept-Encoding\r\nContent-Encoding: gzip\r\nCache-Control: max-age=669\r\nExpires: Mon, 13 Nov 2023 22:02:56 GMT" ⋯ 71617 bytes ⋯ "\"https://www.weather.gov/about\">About Us</a></li>\n                    <li><a href=\"https://www.weather.gov/careers\">Career Opportunities</a></li>\n                </ul>\n            </div>\n        </div>\n        \n            </footer>\n        </main>\n    </body>\n</html>\n"

In [57]:
#parse the HTML data
parsed_html = parsehtml(response)

#Cascadia to select the element with the desired class
selector = Selector(".myforecast-current-lrg")
elements = eachmatch(selector, parsed_html.root)

1-element Vector{HTMLNode}:
 HTMLElement{:p}:<p class="myforecast-current-lrg">
  72°F
</p>



In [59]:
temperature_text = text(elements[1])

"72°F"